<a href="https://colab.research.google.com/github/wayne-chen1887/aps360-project/blob/Generator/Generator_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import random
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt

In [5]:
# link to your Google Drive
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive/Colab_Notebooks/DCGAN/data/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
dataset = dset.ImageFolder(path,
                           transform=transforms.Compose([
                               transforms.Resize(size=(512, 512)),
                               transforms.ToTensor()
                           ]))
# Create the dataloader
dataloader = torch.utils.data.DataLoader(dataset, batch_size=1,)

dataiter = iter(dataloader)
sample = dataiter.next()
sample = sample[0]

In [20]:
# ======================== Not using symmetrical architecture (no dilation in conv transpose layer) ========================
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels=3,  out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(True),
            nn.Conv2d(in_channels=32,  out_channels=64, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.Conv2d(in_channels=64,  out_channels=128, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.Conv2d(in_channels=128,  out_channels=256, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
        )

        self.dilation = nn.Sequential(
            nn.Conv2d(in_channels=256,  out_channels=256, kernel_size=2, stride=1, padding=1, dilation=4),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.Conv2d(in_channels=256,  out_channels=256, kernel_size=2, stride=1, padding=1, dilation=8),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.Conv2d(in_channels=256,  out_channels=256, kernel_size=2, stride=1, padding=1, dilation=16),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(in_channels=256,  out_channels=128, kernel_size=3, stride=3, padding=1, output_padding=2),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(in_channels=128,  out_channels=64, kernel_size=3, stride=2, padding=0, output_padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(in_channels=64,  out_channels=32, kernel_size=3, stride=2, padding=0, output_padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(True),
            nn.ConvTranspose2d(in_channels=32,  out_channels=3, kernel_size=3, stride=1, padding=0, output_padding=0),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        print("Encoder:", x.shape)
        x = self.dilation(x)
        print("Dilation:", x.shape)
        x = self.decoder(x)
        print("Decoder:", x.shape)
        return x

model = Generator()
output = model(sample)
#print("Input:", sample.shape)
#print("Output:", output.shape)

Encoder: torch.Size([1, 256, 64, 64])
Dilation: torch.Size([1, 256, 42, 42])
Decoder: torch.Size([1, 3, 512, 512])


In [21]:
# ======================== Using symmetrical architecture: (dilation in conv transpose layer) ========================
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.encoder = nn.Sequential(
            nn.Conv2d(in_channels=3,  out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(True),
            nn.Conv2d(in_channels=32,  out_channels=64, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.Conv2d(in_channels=64,  out_channels=128, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.Conv2d(in_channels=128,  out_channels=256, kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
        )

        self.dilation = nn.Sequential(
            nn.Conv2d(in_channels=256,  out_channels=256, kernel_size=2, stride=1, padding=1, dilation=4),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.Conv2d(in_channels=256,  out_channels=256, kernel_size=2, stride=1, padding=1, dilation=8),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.Conv2d(in_channels=256,  out_channels=256, kernel_size=2, stride=1, padding=1, dilation=16),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
        )

        self.dilationTranspose = nn.Sequential(
            nn.ConvTranspose2d(in_channels=256,  out_channels=256, kernel_size=2, stride=1, padding=1, dilation=16),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(in_channels=256,  out_channels=256, kernel_size=2, stride=1, padding=1, dilation=8),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(in_channels=256,  out_channels=256, kernel_size=2, stride=1, padding=1, dilation=4),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(in_channels=256,  out_channels=128, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(in_channels=128,  out_channels=64, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(in_channels=64,  out_channels=32, kernel_size=3, stride=2, padding=1, output_padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(True),
            nn.ConvTranspose2d(in_channels=32,  out_channels=3, kernel_size=3, stride=1, padding=1, output_padding=0),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        print("Encoder:", x.shape)
        x = self.dilation(x)
        print("Dilation:", x.shape)
        x = self.dilationTranspose(x)
        print("Dilation Transpose:", x.shape)
        x = self.decoder(x)
        print("Decoder:", x.shape)
        return x

model = Generator()
output = model(sample)
#print("Input:", sample.shape)
#print("Output:", output.shape)

Encoder: torch.Size([1, 256, 64, 64])
Dilation: torch.Size([1, 256, 42, 42])
Dilation Transpose: torch.Size([1, 256, 64, 64])
Decoder: torch.Size([1, 3, 512, 512])
